In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA med")

In [0]:
import requests
import json
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

In [0]:
spark = SparkSession.builder.getOrCreate() # create a spark session
current_time = datetime.utcnow() # get the current time
openFDA_URL = "https://api.fda.gov/drug/label.json"
LIMIT = 100 # max per request
MAX_PAGES = 20

In [0]:
records = []
skip = 0

for page in range(MAX_PAGES):
    params = {
        "search": "openfda.brand_name:*", # search for all records with a brand name
        "limit": LIMIT,
        "skip": skip
    }

    response = requests.get(openFDA_URL, params=params)
    if response.status_code == 404:
        break

    response.raise_for_status()
    data = response.json() # parse the JSON response
    results = data.get("results", []) # get list of results
    
    if not results:
        break

    for result in results:
        product = result.get("openfda", {})
        brand_name = ", ".join(product.get("brand_name", []))
        generic_name = ", ".join(product.get("generic_name", []))
        title = brand_name or generic_name or "Unknown" # use brand name if available, otherwise generic name, otherwise "Unknown
        
        synonyms_list = []
        if brand_name:
            synonyms_list.append(brand_name)
        if generic_name:
            synonyms_list.append(generic_name)
        synonyms = "; ".join(sorted(set(synonyms_list))) if synonyms_list else None

        section_text = []
        for key in [
            "indications_and_usage",
            "dosage_and_administration",
            "warnings",
            "adverse_reactions",
            "contraindications",
        ]:
            values = result.get(key) # get field value from current result

            # extend values depending on if returned as list or string
            if isinstance(values, list):
                section_text.extend(values)
            elif isinstance(values, str):
                section_text.append(values)
        
        # join all text sections
        raw_text = "\n\n".join(section_text) if section_text else None
        
        # build the record dictionary
        record = {
            "doc_id": f"openfda_{result.get('id')}",
            "category": "drug",
            "source": "openfda",
            "title": title,
            "synonyms": synonyms,
            "url": None,
            "raw_text": raw_text,
            "meta_json": json.dumps(result),
            "ingested_at": current_time,
        }

        records.append(record)

    skip += LIMIT

len(records)

In [0]:
# define schema
schema = StructType([
    StructField("doc_id", StringType(), False),
    StructField("category", StringType(), False),
    StructField("source", StringType(), False),
    StructField("title", StringType(), True),
    StructField("synonyms", StringType(), True),
    StructField("url", StringType(), True),
    StructField("raw_text", StringType(), True),
    StructField("meta_json", StringType(), True),
    StructField("ingested_at", TimestampType(), False),
])

df_openfda = spark.createDataFrame(records, schema=schema)

In [0]:
display(df_openfda.limit(5))

In [0]:
spark.sql("""
ALTER TABLE workspace.med.raw_docs_openfda
ADD COLUMNS (
  synonyms STRING,
  url STRING
)
""")


In [0]:
df_openfda.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.med.raw_docs_openfda")